In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Google_ML_Camp

/content/drive/MyDrive/Google_ML_Camp


In [3]:
!pwd

/content/drive/MyDrive/Google_ML_Camp


In [4]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [5]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 876.6/876.6 kB 17.6 MB/s eta 0:00:00


In [6]:
from ultralytics import YOLO

# YOLOv8 세그멘테이션 모델 로드 (YOLOv8n-seg)
model = YOLO('yolov8n-seg.pt')

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# Colab에 Ultralytics 설치
!pip install ultralytics

# 필요한 라이브러리 불러오기
import torch
import torch.nn as nn
from torchvision.ops import DeformConv2d
from ultralytics import YOLO

# Deformable Convolution 모듈 정의
class DeformableConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=1):
        super(DeformableConv2d, self).__init__()
        self.offset_conv = nn.Conv2d(in_channels, 2 * kernel_size * kernel_size,
                                     kernel_size=kernel_size, stride=stride, padding=padding)
        self.deform_conv = DeformConv2d(in_channels, out_channels, kernel_size=kernel_size,
                                        stride=stride, padding=padding, bias=False)

    def forward(self, x):
        offset = self.offset_conv(x)
        out = self.deform_conv(x, offset)
        return out

# DFPN 모듈 정의
class DFPN(nn.Module):
    def __init__(self, in_channels_list, out_channels):
        super(DFPN, self).__init__()
        self.convs = nn.ModuleList()
        for in_channels in in_channels_list:
            self.convs.append(nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0))

    def forward(self, features):
        out_features = []
        for i, feature in enumerate(features):
            out = self.convs[i](feature)
            if i > 0:
                out = out + out_features[i - 1]  # 이전 레벨의 특징과 결합
            out_features.append(out)
        return out_features

# Custom YOLOv8 모델 정의
class CustomYOLOv8Seg(YOLO):
    def __init__(self, model_path='yolov8n-seg.pt'):
        super().__init__(model_path)
        # 기존 백본 교체
        self.model.model[0] = CustomBackbone()  # YOLOv8의 첫 번째 계층이 백본임
        self.model.model[1] = DFPN([64, 128, 256], 256)  # YOLOv8의 두 번째 계층이 넥임

    def forward(self, x):
        return self.model(x)

# 수정된 백본 정의
class CustomBackbone(nn.Module):
    def __init__(self):
        super(CustomBackbone, self).__init__()
        # 간단한 Custom Backbone 정의
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.deform_conv = DeformableConv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.deform_conv(x)
        x = self.conv2(x)
        return x

# 커스텀 모델 학습 및 평가 함수
def train_custom_model():
    # 모델 인스턴스 생성
    model = CustomYOLOv8Seg()

    # 데이터셋 경로 설정
    dataset_path = '/content/drive/MyDrive/Google_ML_Camp/dmg_dup2_copy.v2i.yolov8/data.yaml'

    # 모델 학습
    model.train(data=dataset_path, epochs=200, imgsz=640)

    # 모델 평가
    model.val(data=dataset_path)

train_custom_model()


Ultralytics 8.3.15 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=/content/drive/MyDrive/Google_ML_Camp/dmg_dup2_copy.v2i.yolov8/data.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train16, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=F

100%|██████████| 755k/755k [00:00<00:00, 12.6MB/s]


Overriding model.yaml nc=80 with nc=9

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 70.7MB/s]


AMP: checks passed ✅


...


159 epochs completed in 0.431 hours.
Optimizer stripped from runs/segment/train16/weights/last.pt, 6.2MB
Optimizer stripped from runs/segment/train16/weights/best.pt, 6.2MB

Validating runs/segment/train16/weights/best.pt...
Ultralytics 8.3.15 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
YOLOv8n-seg summary (fused): 213 layers, 2,938,539 parameters, 0 gradients, 10.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  5.94it/s]


                   all         24         60      0.772      0.347      0.441      0.298      0.725      0.336      0.411      0.253
           back_ripped          5          7      0.759      0.143      0.218      0.109      0.757      0.143      0.173     0.0788
             back_wear          6         11          1      0.115      0.403      0.208          1      0.115      0.394      0.114
          front_folded          6         10      0.822        0.2      0.218     0.0569      0.411        0.1     0.0531    0.00659
          front_ripped          5          8          0          0     0.0943     0.0458          0          0     0.0231     0.0109
            front_wear          4         10          1          0      0.081     0.0311          1          0     0.0984     0.0268
           side_ripped          2          2      0.983          1      0.995      0.796      0.982          1      0.995      0.713
             side_wear          3          3          1          0   

val: Scanning /content/drive/MyDrive/Google_ML_Camp/dmg_dup2_copy.v2i.yolov8/valid/labels.cache... 24 images, 0 backgrounds, 0 corrupt: 100%|██████████| 24/24 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.82it/s]


                   all         24         60       0.77      0.347      0.441      0.299      0.715      0.333      0.405       0.25
           back_ripped          5          7      0.754      0.143      0.218      0.108      0.767      0.143      0.173     0.0788
             back_wear          6         11          1      0.116      0.396      0.209      0.874     0.0909      0.346     0.0947
          front_folded          6         10       0.82        0.2      0.219     0.0571      0.412        0.1     0.0531     0.0066
          front_ripped          5          8          0          0     0.0975     0.0461          0          0     0.0229     0.0109
            front_wear          4         10          1          0     0.0811     0.0307          1          0     0.0985     0.0268
           side_ripped          2          2      0.981          1      0.995      0.796      0.988          1      0.995      0.713
             side_wear          3          3          1          0   

# close_mosaic=0, cos_ls=True 추

# front_wear 제거한 데이터셋

In [7]:
# Colab에 Ultralytics 설치
!pip install ultralytics

# 필요한 라이브러리 불러오기
import torch
import torch.nn as nn
from torchvision.ops import DeformConv2d
from ultralytics import YOLO

# Deformable Convolution 모듈 정의
class DeformableConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=1):
        super(DeformableConv2d, self).__init__()
        self.offset_conv = nn.Conv2d(in_channels, 2 * kernel_size * kernel_size,
                                     kernel_size=kernel_size, stride=stride, padding=padding)
        self.deform_conv = DeformConv2d(in_channels, out_channels, kernel_size=kernel_size,
                                        stride=stride, padding=padding, bias=False)

    def forward(self, x):
        offset = self.offset_conv(x)
        out = self.deform_conv(x, offset)
        return out

# DFPN 모듈 정의
class DFPN(nn.Module):
    def __init__(self, in_channels_list, out_channels):
        super(DFPN, self).__init__()
        self.convs = nn.ModuleList()
        for in_channels in in_channels_list:
            self.convs.append(nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0))

    def forward(self, features):
        out_features = []
        for i, feature in enumerate(features):
            out = self.convs[i](feature)
            if i > 0:
                out = out + out_features[i - 1]  # 이전 레벨의 특징과 결합
            out_features.append(out)
        return out_features

# Custom YOLOv8 모델 정의
class CustomYOLOv8Seg(YOLO):
    def __init__(self, model_path='yolov8n-seg.pt'):
        super().__init__(model_path)
        # 기존 백본 교체
        self.model.model[0] = CustomBackbone()  # YOLOv8의 첫 번째 계층이 백본임
        self.model.model[1] = DFPN([64, 128, 256], 256)  # YOLOv8의 두 번째 계층이 넥임

    def forward(self, x):
        return self.model(x)

# 수정된 백본 정의
class CustomBackbone(nn.Module):
    def __init__(self):
        super(CustomBackbone, self).__init__()
        # 간단한 Custom Backbone 정의
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.deform_conv = DeformableConv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.deform_conv(x)
        x = self.conv2(x)
        return x

# 커스텀 모델 학습 및 평가 함수
def train_custom_model():
    # 모델 인스턴스 생성
    model = CustomYOLOv8Seg()

    # 데이터셋 경로 설정
    dataset_path = '/content/drive/MyDrive/Google_ML_Camp/dmg_dup2_copy_wear_x.v1i.yolov8/data.yaml'

    # 모델 학습
    model.train(data=dataset_path, epochs=200, imgsz=640)


    # 모델 평가
    model.val(data=dataset_path)

train_custom_model()


Ultralytics 8.3.17 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=/content/drive/MyDrive/Google_ML_Camp/dmg_dup2_copy_wear_x.v1i.yolov8/data.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train17, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save

100%|██████████| 755k/755k [00:00<00:00, 14.8MB/s]


Overriding model.yaml nc=80 with nc=8

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

...

 ··········


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:


Abort: 